# Abordagem Inicial 

## Objetivo 

### Propor método para:
* encontrar representações comuns de diferentes modalidades sensoriais
* aumentar a quantidade de dados utilizados para redução de dimensionalidade do sinal
* melhorar a sensibilidade de análises MVPA posteriores 

### Inicialmente codificar cada instante de tempo (sem usar janelas no tempo)
### Decodificar o momento que estava na mesma e na outra modalidade a partir do estado reduzido
* Comparar com decodificação usando: (1) Sinais originais, (2) PCA geral, (3) PCA em cada ponto, (4) Auto-encoder
### Ver sobre use da auto-encoders e VAEs
### Checar sobre predição da reprodução do intervalo temporal


In [1]:
import pandas as pd

In [2]:
PATH_FOLDER = '../data/processed/' 

In [3]:
aud_1 =  pd.read_csv(PATH_FOLDER+'auditory_exposure_1.csv')
aud_2 =  pd.read_csv(PATH_FOLDER+'auditory_exposure_2.csv') 
vis_1 =  pd.read_csv(PATH_FOLDER+'visual_exposure_1.csv') 
vis_2 =  pd.read_csv(PATH_FOLDER+'visual_exposure_2.csv')

### Passo 1: treinar com dados completos capturados do experimento
* Entrada e saída como o mesmo sinal (um número para cada eletrodo)
* Usar uma rede fully connected e auto-encoder padrão
    * 64(E) - 32 - 16 - 8 - 16 - 32 - 64(S)
    * Fazer testes verificando o erro de reconstrução
* Uma rede por voluntário e por modalidade
* Pouca quantidade de dados -> incluir regularização, como dropout, para evitar overfitting


# Por voluntário

In [4]:
import sys
sys.path.append("../src/models") 

In [5]:
from my_autoenconder import *

In [6]:
from tqdm.autonotebook import tqdm
import numpy as np
from sklearn.decomposition import PCA
from tensorflow.keras import regularizers
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import LeaveOneOut


/home/brain/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """Entry point for launching an IPython kernel.


In [11]:
def classification_autoEncoder(dataFrame, categoria, exposure):
    '''






    '''

    dataFrame_result = []
    history = []

    for ind, pearson in tqdm(dataFrame.groupby('people')):

        X = pearson.drop(['trial', 'group', 'people'], 1)
        y = pearson['group']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        

        train_mean = np.average(X_train, axis=0)

        X_train_without_mean = np.subtract(X_train, train_mean)
        X_test_without_mean = np.subtract(X_test, train_mean)

        X_train_without_mean = X_train_without_mean.T
        X_test_without_mean = X_test_without_mean.T

        clf = GaussianNB()

        autoEncoder_ = my_autoenconder(epochs=1000,
                                       batch_size=32,
                                       validation_size=0.2,
                                       random_state=42,
                                       regularizer=regularizers.l1(10e-5),
                                       exposure=exposure,
                                       modality=categoria)

        autoEncoder_.make_auto_enconder(16)

        autoEncoder_.fit(X_train_without_mean)

        X_train_auto = autoEncoder_.transform(X_train_without_mean)

        X_test_auto = autoEncoder_.transform(X_test_without_mean)


        clf = clf.fit(X_train_auto, y_train)

        y_pred = clf.predict(X_test_auto)

        dataFrame_result.append([ind,y_pred,y_test.values,categoria,exposure])
        
            
    return dataFrame_result
    #, autoEncoder_.history

In [12]:
resu_aud_1 = classification_autoEncoder(aud_1,'Auditory','E1')


KeyboardInterrupt: 

In [ ]:
resu_aud_2 = classification_autoEncoder(aud_2,'Auditory','E2')


In [ ]:
resu_vis_1 = classification_autoEncoder(vis_1,'Auditory','E1')


In [ ]:
resu_vis_2 = classification_autoEncoder(vis_2,'Auditory','E2')



### Passo 2: gerar uma representação comum entre ambas as modalidades
* Treinar com a mesma rede para visual e auditivo
* Objetivo é verificar se gera uma melhor representação comum entre as modalidades
* Comparar o desempenho com a rede treinada em apenas uma modalidade
* Treinar redes separadas para visual e auditivo, compartilhando a parte da codificação
* Treinamento com exemplos de ambas as modalidades
* Pode ser mais interessante, pois enfatiza uma codificação comum, mas permite diferenciações na decodificação

### Passo 3: obter representações comuns entre sujeitos
* Treinar a mesma rede para todos os participantes 
* Objetivo é gerar as codificações a partir de grandes quantidades de dados
* Abordagem 1: Fazer para a rede uma rede codificação/decodificação comum a todos
    * Depois pode ser afinada para cada participante individualmente
* Abordagem 2: Usar apenas a parte de codificação em comum
    * Enfatiza uma codificação comum, mas permite diferenciações na decodificação
* Neste caso o classificador poderia ser comum também?

Passo 4: capturar informação temporal (opcional)
Repetir os passos anteriores, mas usando 5 valores para cada eletrodo (1 a cada 25 ms)
Objetivo é verificar se mais detalhes temporais melhoram a representação do tempo
Inicialmente utilizar uma arquitetura fully connected 
320(E) - 160 - 80 - 40 - 80 - 160 - 320(S)
Outra possibilidade é colocar alguma estrutura
Transformação é inicialmente realizada sobre a série temporal de cada eletrodo
Poderia também juntar sinais de eletrodos vizinhos
Isso permitiria diminuir o número de conexões